In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [3]:
def remove_duplicate_files(sig_files,bkg_files,bkg_path,aa_run,aa_label,ac_run,ac_label):
    for f in sig_files:
        r = int(f.split("/")[-1].split("_")[-2])
        sr = f.split("/")[-1].split("_")[-1].split(".")[0]
        run_set=""
        if(r==aa_run):
            run_set=aa_label
        if(r==ac_run):
            run_set=ac_label
    
        fname = "%s_%s/sampler_hist_%s.root"%(bkg_path,run_set,sr)
        try:
            bkg_files.remove(fname)
        except:
            print("%s not found. (%s)"%(fname,f))
            sig_files.remove(f)
    all_files = sig_files + bkg_files
    return all_files,sig_files

In [2]:
root_filenames_Set1Run1_Sigs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_SignalFiles/*.root")
root_filenames_Set1Run1_Bkgs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_A[A,C]/*.root")
root_filenames_Set1Run3_Sigs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3_SignalFiles/*.root")
root_filenames_Set1Run3_Bkgs = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3b_A[B,C]/*.root")

In [4]:
root_filenames_Set1Run1,root_filenames_Set1Run1_Sigs = remove_duplicate_files(root_filenames_Set1Run1_Sigs,
                                                                              root_filenames_Set1Run1_Bkgs,
                                                                              "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1",
                                                                              6693842,"AA",6693859,"AC")
root_filenames_Set1Run3,root_filenames_Set1Run3_Sigs = remove_duplicate_files(root_filenames_Set1Run3_Sigs,
                                                                              root_filenames_Set1Run3_Bkgs,
                                                                              "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run3_Sampler_Hists_Set1/Run3b",
                                                                              7165574,"AB",7165592,"AC")

/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_AC/sampler_hist_-1.root not found. (/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Run1_Sampler_Hists_Set1/Run1_SignalFiles/sampler_hist_6693859_-1.root)


In [5]:
root_filenames_Set1 = root_filenames_Set1Run1+root_filenames_Set1Run3
root_filenames_Set2 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/*/*/sampler*.root")
root_filenames_Set3 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/*/*/sampler*.root")
root_filenames_Set4 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set4/*/*/sampler*.root")

In [8]:
def get_id_df(root_filenames):
    id_df = []

    file_count = 0
    event_count = 0
    print("Processing %d files" % len(root_filenames))

    for root_filename in root_filenames:
    
        try:
            id_df.append(uproot.open(root_filename)['generator/id_tree'].pandas.df())
        except:
            print("\tFile %s, No ID Tree. Skipping....")
            
        event_count += len(id_df[-1])
        file_count += 1
        if file_count%500==0:
            print("\tProcessed %d files. %d events processed." % (file_count,event_count))

    id_df = pd.concat(id_df)

    print("Have dataframe objects. Total events is %d." % len(id_df))
    
    return id_df

In [13]:
def merge_on_run_subrun(df,rs_df):
    index_df = pd.MultiIndex.from_arrays([df.index.get_level_values('run_new').array,
                                          df.index.get_level_values('subrun_new').array])
    index_rs = pd.MultiIndex.from_arrays([rs_df[col] for col in ['run', 'subrun']])
    return df.loc[index_df.isin(index_rs)]

In [9]:
id_df_s1 = get_id_df(root_filenames_Set1)

Processing 28537 files
	Processed 500 files. 8276 events processed.
	Processed 1000 files. 13609 events processed.
	Processed 1500 files. 18966 events processed.
	Processed 2000 files. 24306 events processed.
	Processed 2500 files. 29691 events processed.
	Processed 3000 files. 35026 events processed.
	Processed 3500 files. 40396 events processed.
	Processed 4000 files. 45578 events processed.
	Processed 4500 files. 50960 events processed.
	Processed 5000 files. 56268 events processed.
	Processed 5500 files. 61706 events processed.
	Processed 6000 files. 67100 events processed.
	Processed 6500 files. 72316 events processed.
	Processed 7000 files. 77650 events processed.
	Processed 7500 files. 82979 events processed.
	Processed 8000 files. 88404 events processed.
	Processed 8500 files. 93984 events processed.
	Processed 9000 files. 99251 events processed.
	Processed 9500 files. 104583 events processed.
	Processed 10000 files. 109957 events processed.
	Processed 10500 files. 119162 event

In [14]:
rs_reco2_set1_files = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Set1_Run1_RSub.txt",
                 "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Set1_Run3_RSub.txt"]
rs_df_reco2 = pd.concat([pd.read_csv(f,names=["run","subrun"],header=None,sep=".") for f in rs_reco2_set1_files])

In [18]:
index_df = pd.MultiIndex.from_arrays([id_df_s1[col] for col in ['run_new', 'subrun_new']])
index_rs = pd.MultiIndex.from_arrays([rs_df_reco2[col] for col in ['run', 'subrun']])
    
id_df_s1_trim = id_df_s1.loc[index_df.isin(index_rs)]

In [20]:
print(len(id_df_s1),len(id_df_s1_trim))

464773 412937


In [64]:
glee_train_1g1p_ncpi0_df = uproot.open("/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g1pMar2020_v4_stage_1_NCpi0Train.root")["singlephoton/vertex_tree"].pandas.df(flatten=False)
glee_train_1g0p_ncpi0_df = uproot.open("/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g0pMar2020_stage_1_NCpi0Train.root")["singlephoton/vertex_tree"].pandas.df(flatten=False)

In [68]:
glee_train_1g1p_sig_df = uproot.open("/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g1pMar2020_v4_stage_1_NCDeltaTrain.root")["singlephoton/vertex_tree"].pandas.df(flatten=False)
glee_train_1g0p_sig_df = uproot.open("/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g0pMar2020_stage_1_NCDeltaTrain.root")["singlephoton/vertex_tree"].pandas.df(flatten=False)

In [96]:
print(len(glee_train_1g1p_ncpi0_df),len(glee_train_1g0p_ncpi0_df))

18531 23405


In [69]:
index_glee_1g1p_ncpi0 = pd.MultiIndex.from_arrays([glee_train_1g1p_ncpi0_df[col] for col in ['run_number','event_number']])
index_glee_1g0p_ncpi0 = pd.MultiIndex.from_arrays([glee_train_1g0p_ncpi0_df[col] for col in ['run_number','event_number']])

index_glee_1g1p_sig = pd.MultiIndex.from_arrays([glee_train_1g1p_sig_df[col] for col in ['run_number','event_number']])
index_glee_1g0p_sig = pd.MultiIndex.from_arrays([glee_train_1g0p_sig_df[col] for col in ['run_number','event_number']])


index_s1 = pd.MultiIndex.from_arrays([id_df_s1_trim[col] for col in ['run_old','event_old']])

In [89]:
index_glee_1g1p_ncpi0_r1 = pd.MultiIndex.from_arrays([glee_train_1g1p_ncpi0_df.query("run_number<10000")[col] for col in ['run_number','event_number']])
index_glee_1g0p_ncpi0_r1 = pd.MultiIndex.from_arrays([glee_train_1g0p_ncpi0_df.query("run_number<10000")[col] for col in ['run_number','event_number']])

index_glee_1g1p_sig_r1 = pd.MultiIndex.from_arrays([glee_train_1g1p_sig_df.query("run_number<10000")[col] for col in ['run_number','event_number']])
index_glee_1g0p_sig_r1 = pd.MultiIndex.from_arrays([glee_train_1g0p_sig_df.query("run_number<10000")[col] for col in ['run_number','event_number']])


index_s1_r1 = pd.MultiIndex.from_arrays([id_df_s1_trim.query("run_old<10000")[col] for col in ['run_old','event_old']])

In [75]:
n_ovlp_ncpi0_1g1p = index_s1.isin(index_glee_1g1p_ncpi0).sum()
n_ovlp_ncpi0_1g0p = index_s1.isin(index_glee_1g0p_ncpi0).sum()

n_ovlp_sig_1g1p = index_s1.isin(index_glee_1g1p_sig).sum()
n_ovlp_sig_1g0p = index_s1.isin(index_glee_1g0p_sig).sum()

In [90]:
n_ovlp_ncpi0_1g1p_r1 = index_s1.isin(index_glee_1g1p_ncpi0_r1).sum()
n_ovlp_ncpi0_1g0p_r1 = index_s1.isin(index_glee_1g0p_ncpi0_r1).sum()

n_ovlp_sig_1g1p_r1 = index_s1.isin(index_glee_1g1p_sig_r1).sum()
n_ovlp_sig_1g0p_r1 = index_s1.isin(index_glee_1g0p_sig_r1).sum()

In [32]:
df_n = pd.read_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs_Aug19.h5","ev_df")

In [34]:
df_n = merge_on_run_subrun(df_n,rs_df_reco2)

414223


In [93]:
df_n = df_n.sort_index()

In [41]:
#apply fiducial volume
MIN_X = 0.
MAX_X = 254.
MIN_Y = -115.
MAX_Y = 115.
MIN_Z = 0.
MAX_Z = 1036.
fid_vol_query = "start_x>%f and start_x<%f and start_y>%f and start_y<%f and start_z>%f and start_z<%f"%(MIN_X,MAX_X,MIN_Y,MAX_Y,MIN_Z,MAX_Z)

In [81]:
n_tot_ncpi0_1p = len(df_n.query("n_pi0>0 and n_p_40MeV>0 and ccnc==1 and %s"%fid_vol_query))
n_tot_ncpi0_0p = len(df_n.query("n_pi0>0 and n_p_40MeV<=0 and ccnc==1 and %s"%fid_vol_query))

n_tot_1g1p = len(df_n.query("n_gamma==1 and n_p_40MeV>0 and ccnc==1 and %s"%fid_vol_query))
n_tot_1g0p = len(df_n.query("n_gamma==1 and n_p_40MeV<=0 and ccnc==1 and %s"%fid_vol_query))

In [94]:
n_tot_ncpi0_1p_r1 = len(df_n.loc[:7000000].query("n_pi0>0 and n_p_40MeV>0 and ccnc==1 and %s"%fid_vol_query))
n_tot_ncpi0_0p_r1 = len(df_n.loc[:7000000].query("n_pi0>0 and n_p_40MeV<=0 and ccnc==1 and %s"%fid_vol_query))

n_tot_1g1p_r1 = len(df_n.loc[:7000000].query("n_gamma==1 and n_p_40MeV>0 and ccnc==1 and %s"%fid_vol_query))
n_tot_1g0p_r1 = len(df_n.loc[:7000000].query("n_gamma==1 and n_p_40MeV<=0 and ccnc==1 and %s"%fid_vol_query))

In [84]:
print("NC pi0, 1g1p: %d / %d overlap (%f%%)"%(n_ovlp_ncpi0_1g1p,n_tot_ncpi0_1p,n_ovlp_ncpi0_1g1p/float(n_tot_ncpi0_1p)*100.))
print("NC pi0, 1g0p: %d / %d overlap (%f%%)"%(n_ovlp_ncpi0_1g0p,n_tot_ncpi0_0p,n_ovlp_ncpi0_1g0p/float(n_tot_ncpi0_0p)*100.))

print("Single Photon, 1g1p: %d / %d overlap (%f%%)"%(n_ovlp_sig_1g1p,n_tot_1g1p,n_ovlp_sig_1g1p/float(n_tot_1g1p)*100.))
print("Single Photon, 1g0p: %d / %d overlap (%f%%)"%(n_ovlp_sig_1g0p,n_tot_1g0p,n_ovlp_sig_1g0p/float(n_tot_1g0p)*100.))

NC pi0, 1g1p: 370 / 5915 overlap (6.255283%)
NC pi0, 1g0p: 542 / 3409 overlap (15.899091%)
Single Photon, 1g1p: 38 / 205 overlap (18.536585%)
Single Photon, 1g0p: 40 / 163 overlap (24.539877%)


In [95]:
print("NC pi0, 1g1p: %d / %d overlap (%f%%)"%(n_ovlp_ncpi0_1g1p_r1,n_tot_ncpi0_1p_r1,n_ovlp_ncpi0_1g1p_r1/float(n_tot_ncpi0_1p_r1)*100.))
print("NC pi0, 1g0p: %d / %d overlap (%f%%)"%(n_ovlp_ncpi0_1g0p_r1,n_tot_ncpi0_0p_r1,n_ovlp_ncpi0_1g0p_r1/float(n_tot_ncpi0_0p_r1)*100.))

print("Single Photon, 1g1p: %d / %d overlap (%f%%)"%(n_ovlp_sig_1g1p_r1,n_tot_1g1p_r1,n_ovlp_sig_1g1p_r1/float(n_tot_1g1p_r1)*100.))
print("Single Photon, 1g0p: %d / %d overlap (%f%%)"%(n_ovlp_sig_1g0p_r1,n_tot_1g0p_r1,n_ovlp_sig_1g0p_r1/float(n_tot_1g0p_r1)*100.))

NC pi0, 1g1p: 141 / 2343 overlap (6.017926%)
NC pi0, 1g0p: 197 / 1318 overlap (14.946889%)
Single Photon, 1g1p: 33 / 164 overlap (20.121951%)
Single Photon, 1g0p: 36 / 122 overlap (29.508197%)


In [59]:
id_df_s1_trim.to_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/id_df_trimmed.h5","id_df")

/Library/Python/3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->bytes,key->block1_values] [items->Index(['dataset_old'], dtype='object')]

  encoding=encoding,


In [87]:
id_df_s1.query("run_old>10000")

,run_new,subrun_new,event_new,run_old,subrun_old,event_old,dataset_old
entry,,,,,,,
0,7165574,7615,1,17195,9,452,b'bnb_nu'
1,7165574,7615,2,17195,9,466,b'bnb_nu'
2,7165574,7615,3,17195,9,495,b'bnb_nu'
3,7165574,7615,4,17195,9,499,b'bnb_nu'
4,7165574,7615,5,17195,9,500,b'bnb_nu'
...,...,...,...,...,...,...,...
19,7165574,7591,20,15808,66,3343,b'bnb_nu'
20,7165574,7591,21,15808,102,5117,b'bnb_nu'
21,7165574,7591,22,15808,102,5126,b'bnb_nu'


In [97]:
glee_1g1p_fd1_presel_df = uproot.open("/Users/wketchum/Data/MicroBooNE/FakeData2020/gLEE/sbnfit_1g1pMar2020_v4_stage_3_FakeDataRedo1.root")["singlephoton/vertex_tree"].pandas.df(flatten=False)

In [99]:
id_df_s1.rename(columns={"run_new":"run_number","subrun_new":"subrun_number","event_new":"event_number"})

,run_number,subrun_number,event_number,run_old,subrun_old,event_old,dataset_old
entry,,,,,,,
0,6693842,6018,1,7010,411,20551,b'bnb_nu'
1,6693842,6018,2,7010,411,20553,b'bnb_nu'
2,6693842,6018,3,7010,411,20557,b'bnb_nu'
3,6693842,6018,4,7010,411,20559,b'bnb_nu'
4,6693842,6018,5,5068,0,1,b'sig_ncdelta'
...,...,...,...,...,...,...,...
19,7165574,7591,20,15808,66,3343,b'bnb_nu'
20,7165574,7591,21,15808,102,5117,b'bnb_nu'
21,7165574,7591,22,15808,102,5126,b'bnb_nu'


In [98]:
glee_1g1p_fd1_presel_df

,run_number,subrun_number,event_number,pot_per_event,pot_per_subrun,number_of_events_in_subrun,genie_spline_weight,genie_CV_tune_weight,photonu_weight_low,photonu_weight_high,...,mctruth_pi0_leading_photon_exiting_TPC,mctruth_pi0_subleading_photon_energy,mctruth_pi0_subleading_photon_end_process,mctruth_pi0_subleading_photon_start,mctruth_pi0_subleading_photon_end,mctruth_pi0_subleading_photon_exiting_TPC,mctruth_exiting_pi0_E,mctruth_exiting_pi0_px,mctruth_exiting_pi0_py,mctruth_exiting_pi0_pz
entry,,,,,,,,,,,,,,,,,,,,,
0,7165574,707,9,0.0,1.988298e+16,-1.766705e-22,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
1,7165574,7668,20,0.0,2.987895e+16,3.486007e-169,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
2,7165574,6742,14,0.0,2.467161e+16,3.106517e-246,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
3,7165574,9471,4,0.0,3.990894e+16,-1.181203e+114,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
4,7165574,6666,8,0.0,2.169559e+16,1.967437e-53,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,6693842,3952,4,0.0,9.270474e+15,1.292598e+244,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
279,6693859,498,7,0.0,2.640785e+16,-1.635142e+76,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]
280,6693842,6931,13,0.0,2.321792e+16,5.651382e-61,1.0,2.887580e-273,9.145465e+115,8.717078e+191,...,-999,-9999.0,b'none',"[-9999.0, -9999.0, -9999.0]","[-9999.0, -9999.0, -9999.0]",-999,[],[],[],[]


In [102]:
glee_1g1p_fd1_presel_df = pd.merge(glee_1g1p_fd1_presel_df, id_df_s1.rename(columns={"run_new":"run_number","subrun_new":"subrun_number","event_new":"event_number"}), how='left', on=["run_number","subrun_number","event_number"])

In [103]:
index_s1_presel = pd.MultiIndex.from_arrays([glee_1g1p_fd1_presel_df[col] for col in ['run_old','event_old']])

In [108]:
print(index_s1_presel.isin(index_glee_1g1p_ncpi0).sum(),len(glee_1g1p_fd1_presel_df),index_s1_presel.isin(index_glee_1g1p_ncpi0).sum()/len(glee_1g1p_fd1_presel_df))
print(index_s1_presel.isin(index_glee_1g0p_ncpi0).sum()/len(glee_1g1p_fd1_presel_df))

164 283 0.5795053003533569
0.0


In [105]:
print(n_ovlp_ncpi0_1g1p_presel)

164


In [109]:
print(index_s1_presel.isin(index_glee_1g1p_sig).sum(),len(glee_1g1p_fd1_presel_df),index_s1_presel.isin(index_glee_1g1p_sig).sum()/len(glee_1g1p_fd1_presel_df))
print(index_s1_presel.isin(index_glee_1g0p_sig).sum()/len(glee_1g1p_fd1_presel_df))

25 283 0.08833922261484099
0.0
